In [ ]:
# --- useful when the package is not yet installed
import sys
sys.path.insert(0, "../")
# ---

# imports
from reform import simu_utils
from simtk.openmm import app
from simtk import unit
import numpy as np

In [ ]:
# simulation setup
interface = "replicated_system" # can be "single_threaded" or "replicated_system"
pdb_path = "../reform/tests/spep_0000.pdb"

loading_NN_force = True
NN_model_path = "../reform/tests/central.pt"

TIME_STEP = 2   # in fs
SIMU_TIME = 50  # in ps
RECORDING_INTERVAL = 1   # in ps
EXCHANGE_INTERVAL = 0.2  # in ps; or 0. when you don't need the exchanges

temps_intended = [300., 350.] # in K

OUTPUT_PATH = "/tmp/00100.npy"
CHECKPOINT_PATH = "/tmp/00100.npz"

In [ ]:
# prepare the MultiTSimulation
n_replicas = len(temps_intended)

# --- routinary OpenMM system setup, here it's for a vacuum system of ala2
pdb = app.PDBFile(pdb_path)
ff = app.ForceField("amber99sbildn.xml")
system = ff.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds,
                         hydrogenMass=4 * unit.amu)
integrator_params = {"integrator": "Langevin", "friction_in_inv_ps": 1.0, "time_step_in_fs": 2.0}
# ---

# loading the OpenMM-Torch plugin
try:
    from openmmtorch import TorchForce
    openmmtorch_loaded = True
except ImportError as e:
    print("Warning: no openmm-torch module found. NN model is not loaded.")
    openmmtorch_loaded = False

# --- setting up the simulation ---
if loading_NN_force and openmmtorch_loaded:
    # load torch force
    f = TorchForce(MODEL_PATH)
    # this difference is caused by the plugin design.
    # Not solvable from the `reform` side.
    if interface == "single_threaded":
        system.addForce(f)
        simu = simu_utils.MultiTSimulation(system, temps_intended, interface=interface, platform="CUDA",
                                           integrator_params=integrator_params, verbose=False)
    elif interface == "replicated_system":
        simu = simu_utils.MultiTSimulation(system, temps_intended, replicated_system_additional_forces=[f],
                                           interface=interface, platform="CUDA",
                                           integrator_params=integrator_params, verbose=False)
else:
    simu = simu_utils.MultiTSimulation(system, temps_intended, interface=interface,
                                       integrator_params=integrator_params, verbose=False)

simu.set_positions([pdb.getPositions()] * n_replicas)
simu.minimize_energy()
simu.set_velocities_to_temp()
simu.run(2000)  # pre-equilibration
_ = simu.reset_step_counter()

In [ ]:
# install the simulation hooks for recording and exchange
simu_steps = simu_utils.recording_hook_setup(simu=simu, simu_time=SIMU_TIME,
                                             recording_interval=RECORDING_INTERVAL,
                                             output_path=OUTPUT_PATH,
                                             exchange_interval=EXCHANGE_INTERVAL)

In [ ]:
# a timed run
from timeit import default_timer as timer
start = timer()
simu.run(int(simu_steps / 2))
stop = timer()
print("Production run of 25 ps takes {:.2f} seconds.".format(stop - start))

In [ ]:
# example of saving and loading checkpoints
simu.save_chkpt(CHECKPOINT_PATH)
simu.load_chkpt(CHECKPOINT_PATH)

In [ ]:
# finishing the reset from loaded state
simu.run(simu_steps - int(simu_steps / 2))

In [ ]:
# saving the traj
simu.save()

In [ ]:
# check the output npy file
# dimensions [N_replicas, N_frames, N_atoms, N_dims=3]
np.load(OUTPUT_PATH).shape